<a href="https://colab.research.google.com/github/BalthazazarDuc/Data-Science-Machine-Learning/blob/Balt/LLM__Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ABC_Classification

In [ ]:
!pip install transformers torch
!pip install accelerate
!pip install transformers[torch] accelerate --upgrade
!python -m spacy download fr_core_news_sm
!pip install transformers datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import spacy
import nltk
from nltk import word_tokenize, pos_tag
from collections import Counter
import pandas as pd
import spacy
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import accelerate
import transformers
from sklearn.model_selection import train_test_split
print(accelerate.__version__)
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import accelerate
import transformers
from sklearn.model_selection import train_test_split
print(accelerate.__version__)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


0.30.1
0.30.1


##Distilbert

In [ ]:
train=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/train_letter.csv')
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')

label_dict = {
    'A': 0,
    'B': 1,
    'C': 2,
}

# Appliquer le mapping sur la colonne des étiquettes
train['label'] = train['letter'].map(label_dict)


In [ ]:
# Initialize the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Tokenization function to encode the texts
def tokenize_function(examples):
    # Tokenize the input text and include the labels for each example
    tokenized_inputs = tokenizer(examples['sentence'], padding="max_length", truncation=True)
    # Add labels to the dictionary under 'labels' key
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

def tokenize_function_test(examples):
    # Tokenize the input text and include the labels for each example
    tokenized_inputs = tokenizer(examples['sentence'], padding="max_length", truncation=True)
    return tokenized_inputs

# Convert DataFrame to Hugging Face dataset format, tokenize and set format
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function_test, batched=True)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Define the model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # use the tokenized train dataset
    eval_dataset=test_dataset     # use the tokenized test dataset for evaluation
)

# Train the model
trainer.train()
trainer.evaluate()

In [ ]:
predictions = trainer.predict(test_dataset)
predicted_labels_indices = predictions.predictions.argmax(-1)
predicted_labels_indices

In [ ]:
# Assuming the label dictionary used earlier was like this:
label_dict = {'A': 0, 'B': 1, 'C': 2}
# Invert the dictionary to map back from indices to labels
index_to_label = {v: k for k, v in label_dict.items()}

# Map the predicted label indices to actual labels
predicted_labels = [index_to_label[idx] for idx in predicted_labels_indices]


In [ ]:
test['difficulty'] = predicted_labels
test

# Prepare the Baltha output for the predictions

In [ ]:
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/Balt/predicted_camambert_4_epoch.csv')
test['difficulty']=test['difficulty_distilbert']
test

,id,sentence,difficulty_distilbert,difficulty
0,0,Nous dûmes nous excuser des propos que nous eû...,C,C
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...,A,A
2,2,"Et, paradoxalement, boire froid n'est pas la b...",B,B
3,3,"Ce n'est pas étonnant, car c'est une saison my...",A,A
4,4,"Le corps de Golo lui-même, d'une essence aussi...",C,C
...,...,...,...,...
1195,1195,C'est un phénomène qui trouve une accélération...,B,B
1196,1196,Je vais parler au serveur et voir si on peut d...,A,A
1197,1197,Il n'était pas comme tant de gens qui par pare...,C,C
1198,1198,Ils deviennent dangereux pour notre économie.,B,B


from matplotlib import pyplot as plt
test['id'].plot(kind='hist', bins=20, title='id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
test.groupby('difficulty_distilbert').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
test.groupby('difficulty').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['id']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'id'}, axis=1)
              .sort_values('id', ascending=True))
  xs = counted['id']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = test.sort_values('id', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('difficulty_distilbert')):
  _plot_series(series, series_name, i)
  fig.legend(title='difficulty_distilbert', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('id')
_ = plt.ylabel('count()')

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['difficulty'].value_counts()
    for x_label, grp in test.groupby('difficulty_distilbert')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('difficulty_distilbert')
_ = plt.ylabel('difficulty')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(test['difficulty_distilbert'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(test, x='id', y='difficulty_distilbert', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(test['difficulty'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(test, x='id', y='difficulty', inner='box', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/ABC/ABC_camambert_5_epoch_8_64_acc343.csv')
test

,id,sentence,difficulty
0,0,Nous dûmes nous excuser des propos que nous eû...,C
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...,B
2,2,"Et, paradoxalement, boire froid n'est pas la b...",B
3,3,"Ce n'est pas étonnant, car c'est une saison my...",A
4,4,"Le corps de Golo lui-même, d'une essence aussi...",C
...,...,...,...
1195,1195,C'est un phénomène qui trouve une accélération...,B
1196,1196,Je vais parler au serveur et voir si on peut d...,A
1197,1197,Il n'était pas comme tant de gens qui par pare...,C
1198,1198,Ils deviennent dangereux pour notre économie.,B


In [ ]:
test.groupby('difficulty').size()

difficulty
A    422
B    476
C    302
dtype: int64

In [ ]:
test_A1 = test[test['difficulty'] == 'A'].drop('difficulty', axis = 1)

In [ ]:
test_B2 = test[test['difficulty'] == 'B'].drop('difficulty', axis = 1)

In [ ]:
test_C2 = test[test['difficulty'] == 'C'].drop('difficulty', axis = 1)

# Training the A1 model LLM

In [ ]:
train_A1=pd.read_csv('https://github.com/BalthazazarDuc/Data-Science-Machine-Learning/raw/main/train_A1.csv')
train_A1['label'] = train_A1['A1'].replace({1: 'A1', 0: 'A2'})
train_A1

,sentence,A1,label
0,Les coûts kilométriques réels peuvent diverger...,0,A2
1,"Le bleu, c'est ma couleur préférée mais je n'a...",1,A1
2,Le test de niveau en français est sur le site ...,1,A1
3,Est-ce que ton mari est aussi de Boston?,1,A1
4,"Dans les écoles de commerce, dans les couloirs...",0,A2
...,...,...,...
4795,"C'est pourquoi, il décida de remplacer les hab...",0,A2
4796,Il avait une de ces pâleurs splendides qui don...,0,A2
4797,"Et le premier samedi de chaque mois, venez ren...",0,A2
4798,Les coûts liés à la journalisation n'étant pas...,0,A2


In [ ]:
train_A1=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')
train_A1 = train_A1[train_A1['difficulty'].isin(['A1', 'A2'])]
train_A1['A1'] = train_A1['difficulty'].replace({ 'A1':1, 'A2':0})
train_A1 = train_A1.reset_index(drop=True)
train_A1

,id,sentence,difficulty,A1
0,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1,1
1,2,Le test de niveau en français est sur le site ...,A1,1
2,3,Est-ce que ton mari est aussi de Boston?,A1,1
3,5,voilà une autre histoire que j'ai beaucoup aimée.,A2,0
4,6,Les médecins disent souvent qu'on doit boire u...,A2,0
...,...,...,...,...
1603,4786,Qu'est-ce que c'est ?,A1,1
1604,4788,Je suis le professeur Duquesnes.,A1,1
1605,4789,C'est exactement l'une des meilleurs methodes ...,A2,0
1606,4790,Merci pour cette histoire et bonne année !,A1,1


In [ ]:
# Charger le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')


# Charger le modèle pré-entraîné
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=2)

# Fonction pour tokeniser les données
def encode(texts):
    return tokenizer(texts.tolist(), max_length=256, padding="max_length", truncation=True)

# Application de la fonction de tokenisation
train_encodings_A1 = encode(train_A1['sentence'])
test_encodings_A1 = encode(test_A1['sentence'])

class FrenchLevelDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Création des instances de dataset
train_dataset_A1 = FrenchLevelDataset(train_encodings_A1, train_A1['A1'])
test_dataset_A1 = FrenchLevelDataset(test_encodings_A1)

# Configuration des paramètres d'entraînement

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Création de l'objet Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_A1,
    eval_dataset=test_dataset_A1  # Vous pouvez utiliser une partie des données train pour la validation, si nécessaire
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Lancement de l'entraînement
trainer.train()

# Évaluation
trainer.evaluate()

Epoch,Training Loss,Validation Loss
1,0.312400,No log
2,0.308600,No log
3,0.153400,No log
4,0.074300,No log


{'eval_runtime': 3.4684,
 'eval_samples_per_second': 133.781,
 'eval_steps_per_second': 2.307,
 'epoch': 4.0}

In [ ]:
predictions = trainer.predict(test_dataset_A1)
predicted_labels_indices = predictions.predictions.argmax(-1)

# Assuming the label dictionary used earlier was like this:
label_dict = {'A1': 1, 'A2': 0}
# Invert the dictionary to map back from indices to labels
index_to_label = {v: k for k, v in label_dict.items()}

# Map the predicted label indices to actual labels
predicted_labels = [index_to_label[idx] for idx in predicted_labels_indices]

In [ ]:
test_A1['difficulty'] = predicted_labels
test_A1

,id,sentence,difficulty_distilbert,difficulty
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...,A,A2
3,3,"Ce n'est pas étonnant, car c'est une saison my...",A,A2
6,6,"Madame, Monsieur, Votre fils Léo arrive tous l...",A,A2
7,7,Comment tu as trouvé le repas de ce midi,A,A2
9,9,"Je ne peux pas vous laisser dire cela, Madame.",A,A2
...,...,...,...,...
1187,1187,L'histoire de ce livre est incroyable,A,A2
1192,1192,Je n'aime pas le pain.,A,A1
1193,1193,"Ici, il fait beau et chaud.",A,A1
1194,1194,Vous faites de l'exercice physique,A,A2


In [ ]:
test_A1.groupby('difficulty').size()

difficulty
A1    166
A2    298
dtype: int64

#Train A1 Pipeline

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler


preprocessor = ColumnTransformer(
    transformers=[
        ('txt', TfidfVectorizer(ngram_range=(1, 2)), 'sentence'),
        ('num', StandardScaler(), ['word_count', 'unique_words', 'comma_count'])  # Standardize numeric features
    ])


train_A1=pd.read_csv('https://github.com/BalthazazarDuc/Data-Science-Machine-Learning/raw/main/train_A1.csv')

train_A1['A1'] = train_A1['A1'].replace({1: 'A1', 0: 'A2'})

nlp = spacy.load('fr_core_news_sm')

def extract_features(sentence):
    doc = nlp(sentence)
    word_count = len([token.text for token in doc])
    unique_words = len(set(token.text for token in doc))
    comma_count = sentence.count(',')


    features = {
        'word_count': word_count,
        'unique_words': unique_words,
        'comma_count': comma_count,
    }

    return features

train_features_A1 = train_A1['sentence'].apply(lambda x: pd.Series(extract_features(x)))
train_A1[train_features_A1.columns] = train_features_A1


X_A1 = train_A1.drop(['A1'], axis=1)  # 'difficulty' is the target feature
y_A1 = train_A1['A1']

test_features_A1 = test_A1['sentence'].apply(lambda x: pd.Series(extract_features(x)))
test_A1[test_features_A1.columns] = test_features_A1

from sklearn.svm import SVC

clf_A1 = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear', max_iter=10000))  # Use SVC with a linear kernel
])

clf_A1.fit(X_A1, y_A1)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('txt',
                                                  TfidfVectorizer(ngram_range=(1,
                                                                               2)),
                                                  'sentence'),
                                                 ('num', StandardScaler(),
                                                  ['word_count', 'unique_words',
                                                   'comma_count'])])),
                ('classifier', SVC(kernel='linear', max_iter=10000))])

In [ ]:
predictions_A1 = clf_A1.predict(test_A1)

In [ ]:
test_A1['difficulty'] = predictions_A1

# Training the B2 model

In [ ]:
train_B2=pd.read_csv('https://github.com/BalthazazarDuc/Data-Science-Machine-Learning/raw/main/train_B2.csv')
train_B2['label'] = train_B2['B2'].replace({1: 'B2', 0: 'B1'})

In [ ]:
train_B2=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')
train_B2 = train_B2[train_B2['difficulty'].isin(['B1', 'B2', 'C1'])]
train_B2['B2'] = train_B2['difficulty'].replace({ 'B1':0, 'B2':1, 'C1':2})
train_B2 = train_B2.reset_index(drop=True)
train_B2

,id,sentence,difficulty,B2
0,0,Les coûts kilométriques réels peuvent diverger...,C1,2
1,4,"Dans les écoles de commerce, dans les couloirs...",B1,0
2,7,Il est particulièrement observé chez les perso...,B2,1
3,9,"Nous allons bien, nous habitons dans une petit...",B1,0
4,11,La presse s'est abondamment fait l'écho des ef...,B2,1
...,...,...,...,...
2380,4792,Les villages situés à mi-côte sont cachés par ...,C1,2
2381,4793,La réduction du dioxyde de carbone par l'eau n...,B2,1
2382,4794,Elle connaissait à présent la petitesse des pa...,C1,2
2383,4795,"C'est pourquoi, il décida de remplacer les hab...",B2,1


In [ ]:
# Charger le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')


# Charger le modèle pré-entraîné
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=3)

# Fonction pour tokeniser les données
def encode(texts):
    return tokenizer(texts.tolist(), max_length=256, padding="max_length", truncation=True)

# Application de la fonction de tokenisation
train_encodings_B2 = encode(train_B2['sentence'])
test_encodings_B2 = encode(test_B2['sentence'])

class FrenchLevelDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Création des instances de dataset
train_dataset_B2 = FrenchLevelDataset(train_encodings_B2, train_B2['B2'])
test_dataset_B2 = FrenchLevelDataset(test_encodings_B2)

# Configuration des paramètres d'entraînement

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Création de l'objet Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_B2,
    eval_dataset=test_dataset_B2  # Vous pouvez utiliser une partie des données train pour la validation, si nécessaire
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Lancement de l'entraînement
trainer.train()

# Évaluation
trainer.evaluate()

Epoch,Training Loss,Validation Loss
1,0.744000,No log
2,0.759900,No log
3,0.445600,No log
4,0.325000,No log
5,0.065800,No log


{'eval_runtime': 3.222,
 'eval_samples_per_second': 147.733,
 'eval_steps_per_second': 2.483,
 'epoch': 5.0}

In [ ]:
predictions = trainer.predict(test_dataset_B2)
predicted_labels_indices = predictions.predictions.argmax(-1)

# Assuming the label dictionary used earlier was like this:
label_dict = {'B1': 0, 'B2': 1, 'C1':2}
# Invert the dictionary to map back from indices to labels
index_to_label = {v: k for k, v in label_dict.items()}

# Map the predicted label indices to actual labels
predicted_labels = [index_to_label[idx] for idx in predicted_labels_indices]

In [ ]:
test_B2['difficulty'] = predicted_labels
test_B2


,id,sentence,difficulty
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...,B1
2,2,"Et, paradoxalement, boire froid n'est pas la b...",B1
8,8,Mais la racine du mal est bel est bien cette f...,C1
12,12,"Pourtant, sur cette distance, le vélo est le m...",B2
16,16,"En tombant, certains arbres ont écrasé des str...",B2
...,...,...,...
1190,1190,"Si elle avait été suffisament investie, elle a...",B2
1191,1191,Il autorise le recours à l'IVG par l'intermédi...,C1
1195,1195,C'est un phénomène qui trouve une accélération...,B1
1198,1198,Ils deviennent dangereux pour notre économie.,B2


In [ ]:
test_B2.groupby('difficulty').size()

difficulty
B1    159
B2    239
C1     78
dtype: int64

# Training the C2 model

In [ ]:
train_C2=pd.read_csv('https://github.com/BalthazazarDuc/Data-Science-Machine-Learning/raw/main/train_C2.csv')
train_C2['label'] = train_C2['C2'].replace({1: 'C2', 0: 'C1'})


In [ ]:
train_C2=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')
train_C2 = train_C2[train_C2['difficulty'].isin(['C1', 'C2'])]
train_C2['C2'] = train_C2['difficulty'].replace({ 'C1':0, 'C2':1})
train_C2 = train_C2.reset_index(drop=True)
train_C2

,id,sentence,difficulty,C2
0,0,Les coûts kilométriques réels peuvent diverger...,C1,0
1,15,Ma timidité me quittait dès que je m'éloignais...,C2,1
2,21,"Sans plus d'hésitation, il tourna vers la gauc...",C2,1
3,22,"Comme le signale l'expression ""lingua franca"",...",C2,1
4,25,Qu'est-ce qui peut bien poser problème dans ce...,C2,1
...,...,...,...,...
1600,4792,Les villages situés à mi-côte sont cachés par ...,C1,0
1601,4794,Elle connaissait à présent la petitesse des pa...,C1,0
1602,4796,Il avait une de ces pâleurs splendides qui don...,C1,0
1603,4798,Les coûts liés à la journalisation n'étant pas...,C2,1


##Distilbert

In [ ]:
# Charger le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')


# Charger le modèle pré-entraîné
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=2)

# Fonction pour tokeniser les données
def encode(texts):
    return tokenizer(texts.tolist(), max_length=256, padding="max_length", truncation=True)

# Application de la fonction de tokenisation
train_encodings_C2 = encode(train_C2['sentence'])
test_encodings_C2 = encode(test_C2['sentence'])

class FrenchLevelDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Création des instances de dataset
train_dataset_C2 = FrenchLevelDataset(train_encodings_C2, train_C2['C2'])
test_dataset_C2 = FrenchLevelDataset(test_encodings_C2)

# Configuration des paramètres d'entraînement

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Création de l'objet Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_C2,
    eval_dataset=test_dataset_C2  # Vous pouvez utiliser une partie des données train pour la validation, si nécessaire
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Lancement de l'entraînement
trainer.train()

# Évaluation
trainer.evaluate()

Epoch,Training Loss,Validation Loss
1,0.667400,No log
2,0.626200,No log
3,0.526400,No log
4,0.399100,No log


{'eval_runtime': 1.84,
 'eval_samples_per_second': 164.128,
 'eval_steps_per_second': 2.717,
 'epoch': 4.0}

In [ ]:
predictions = trainer.predict(test_dataset_C2)
predicted_labels_indices = predictions.predictions.argmax(-1)

# Assuming the label dictionary used earlier was like this:
label_dict = {'C1': 0, 'C2': 1}
# Invert the dictionary to map back from indices to labels
index_to_label = {v: k for k, v in label_dict.items()}

# Map the predicted label indices to actual labels
predicted_labels = [index_to_label[idx] for idx in predicted_labels_indices]

##Camemebert

In [ ]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd


# Initialize the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Tokenization function to encode the texts
def tokenize_function(examples):
    # Tokenize the input text and include the labels for each example
    tokenized_inputs = tokenizer(examples['sentence'], padding="max_length", truncation=True)
    # Add labels to the dictionary under 'labels' key
    tokenized_inputs['labels'] = examples['C2']
    return tokenized_inputs

def tokenize_function_test(examples):
    # Tokenize the input text and include the labels for each example
    tokenized_inputs = tokenizer(examples['sentence'], padding="max_length", truncation=True)
    return tokenized_inputs

# Convert DataFrame to Hugging Face dataset format, tokenize and set format
train_dataset = Dataset.from_pandas(train_C2)
test_dataset = Dataset.from_pandas(test_C2)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function_test, batched=True)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Define the model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/1605 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # use the tokenized train dataset
    eval_dataset=test_dataset     # use the tokenized test dataset for evaluation
)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.603100,No log
2,0.573800,No log
3,0.413200,No log
4,0.088600,No log
5,0.102300,No log


TrainOutput(global_step=1005, training_loss=0.3947095634035803, metrics={'train_runtime': 770.55, 'train_samples_per_second': 10.415, 'train_steps_per_second': 1.304, 'total_flos': 2111466219264000.0, 'train_loss': 0.3947095634035803, 'epoch': 5.0})

In [ ]:
predictions = trainer.predict(test_dataset)
predicted_labels_indices = predictions.predictions.argmax(-1)

# Assuming the label dictionary used earlier was like this:
label_dict = {'C1': 0, 'C2': 1}
# Invert the dictionary to map back from indices to labels
index_to_label = {v: k for k, v in label_dict.items()}

# Map the predicted label indices to actual labels
predicted_labels = [index_to_label[idx] for idx in predicted_labels_indices]


##Predicts

In [ ]:
test_C2['difficulty'] = predicted_labels
test_C2

,id,sentence,difficulty
0,0,Nous dûmes nous excuser des propos que nous eû...,C2
4,4,"Le corps de Golo lui-même, d'une essence aussi...",C2
5,5,"Elle jeta un cri, un petit cri, voulut se dres...",C1
13,13,"Jeune retraité de l'éducation nationale, Rog...",C1
20,20,"Et, dans le mirage confus où s'égaraient ses e...",C2
...,...,...,...
1164,1164,Le ministre de l'Instruction publique cherchai...,C1
1174,1174,Le ressortissants d'un pays tiers à l'Union eu...,C1
1180,1180,"Mais soyons sérieux, nouvelles en bref : écono...",C1
1186,1186,"En partenariat avec l'INPES, elle a mis en pla...",C1


In [ ]:
test_C2.groupby('difficulty').size()

difficulty
C1    156
C2    146
dtype: int64

# Sub-df's for predicting

## Big predictions frame

In [ ]:


#result_extended.to_csv('result_pipeline.csv', index=False)
result_ABC = pd.concat([test_A1, test_B2, test_C2], ignore_index=True)

### select relevant columns

In [ ]:
result_ABC = result_ABC[['id', 'difficulty', 'sentence']]

### in ascending order

In [ ]:
result_ABC = result_ABC.sort_values(by='id', ascending=True)

## dataframe for further prediction

In [ ]:
result_A2B1C1 = result_ABC[result_ABC['difficulty'].isin(['A2', 'B1', 'C1'])]
#result_A2B1C1['difficulty']='D1'
result_A2B1C1

,id,difficulty,sentence
422,1,B1,Vous ne pouvez pas savoir le plaisir que j'ai ...
423,2,B1,"Et, paradoxalement, boire froid n'est pas la b..."
1,6,A2,"Madame, Monsieur, Votre fils Léo arrive tous l..."
2,7,A2,Comment tu as trouvé le repas de ce midi
424,8,C1,Mais la racine du mal est bel est bien cette f...
...,...,...,...
894,1191,C1,Il autorise le recours à l'IVG par l'intermédi...
420,1194,A2,Vous faites de l'exercice physique
895,1195,B1,C'est un phénomène qui trouve une accélération...
421,1196,A2,Je vais parler au serveur et voir si on peut d...


### dataframe (on this no more work needed)

In [ ]:
result_A1B2C2 = result_ABC[~result_ABC['difficulty'].isin(['A2', 'B1', 'C1'])]
result_A1B2C2


,id,difficulty,sentence
898,0,C2,Nous dûmes nous excuser des propos que nous eû...
0,3,A1,"Ce n'est pas étonnant, car c'est une saison my..."
899,4,C2,"Le corps de Golo lui-même, d'une essence aussi..."
900,5,C2,"Elle jeta un cri, un petit cri, voulut se dres..."
3,9,A1,"Je ne peux pas vous laisser dire cela, Madame."
...,...,...,...
893,1190,B2,"Si elle avait été suffisament investie, elle a..."
418,1192,A1,Je n'aime pas le pain.
419,1193,A1,"Ici, il fait beau et chaud."
1199,1197,C2,Il n'était pas comme tant de gens qui par pare...


#A1B2C2 Classification with distilbert




In [ ]:
train=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')

label_dict = {
    'A1': 0,
    'A2': 1,
    'B1': 2,
    'B2': 3,
    'C1': 4,
    'C2': 5
}

# Appliquer le mapping sur la colonne des étiquettes
train['difficulty'] = train['difficulty'].map(label_dict)

In [ ]:
# Charger le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')


# Charger le modèle pré-entraîné
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Fonction pour tokeniser les données
def encode(texts):
    return tokenizer(texts.tolist(), max_length=256, padding="max_length", truncation=True)

# Application de la fonction de tokenisation
train_encodings = encode(train['sentence'])
test_encodings = encode(result_A2B1C1['sentence'])

In [ ]:
class FrenchLevelDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Création des instances de dataset
train_dataset = FrenchLevelDataset(train_encodings, train['difficulty'])
test_dataset = FrenchLevelDataset(test_encodings)

In [ ]:
# Charger le modèle pré-entraîné
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)  # Assurez-vous que le nombre de labels correspond au nombre de niveaux de difficulté

# Configuration des paramètres d'entraînement

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Création de l'objet Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset  # Vous pouvez utiliser une partie des données train pour la validation, si nécessaire
)

# Lancement de l'entraînement
trainer.train()

# Évaluation
trainer.evaluate()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,1.214100,No log
2,1.178500,No log
3,0.742700,No log
4,0.577400,No log


{'eval_runtime': 5.215,
 'eval_samples_per_second': 138.638,
 'eval_steps_per_second': 2.301,
 'epoch': 4.0}

In [ ]:
# Generate predictions for the test dataset
predictions = trainer.predict(test_dataset)
predicted_labels_indices = predictions.predictions.argmax(-1)

In [ ]:
# Assuming the label dictionary used earlier was like this:
label_dict = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
# Invert the dictionary to map back from indices to labels
index_to_label = {v: k for k, v in label_dict.items()}

# Map the predicted label indices to actual labels
predicted_labels = [index_to_label[idx] for idx in predicted_labels_indices]

In [ ]:

# Assuming the test DataFrame is named 'test' and has a column 'sentence' for the input text
result_A2B1C1['difficulty'] = predicted_labels
result_A2B1C1

<ipython-input-60-11c8f1f88154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_A2B1C1['difficulty'] = predicted_labels


,id,difficulty,sentence
422,1,A2,Vous ne pouvez pas savoir le plaisir que j'ai ...
423,2,B1,"Et, paradoxalement, boire froid n'est pas la b..."
1,6,A2,"Madame, Monsieur, Votre fils Léo arrive tous l..."
2,7,A2,Comment tu as trouvé le repas de ce midi
424,8,C2,Mais la racine du mal est bel est bien cette f...
...,...,...,...
894,1191,C1,Il autorise le recours à l'IVG par l'intermédi...
420,1194,A2,Vous faites de l'exercice physique
895,1195,B1,C'est un phénomène qui trouve une accélération...
421,1196,A2,Je vais parler au serveur et voir si on peut d...


#A1B2C2 Classification with camembert



In [ ]:
train=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')

label_dict = {
    'A1': 0,
    'A2': 1,
    'B1': 2,
    'B2': 3,
    'C1': 4,
    'C2': 5
}

# Appliquer le mapping sur la colonne des étiquettes
train['label'] = train['difficulty'].map(label_dict)

In [ ]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd


# Initialize the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Tokenization function to encode the texts
def tokenize_function(examples):
    # Tokenize the input text and include the labels for each example
    tokenized_inputs = tokenizer(examples['sentence'], padding="max_length", truncation=True)
    # Add labels to the dictionary under 'labels' key
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

def tokenize_function_test(examples):
    # Tokenize the input text and include the labels for each example
    tokenized_inputs = tokenizer(examples['sentence'], padding="max_length", truncation=True)
    return tokenized_inputs

# Convert DataFrame to Hugging Face dataset format, tokenize and set format
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(result_A2B1C1)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function_test, batched=True)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Define the model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

Map:   0%|          | 0/826 [00:00<?, ? examples/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # use the tokenized train dataset
    eval_dataset=test_dataset     # use the tokenized test dataset for evaluation
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.244100,No log
2,1.022600,No log
3,0.741700,No log
4,0.563200,No log


TrainOutput(global_step=1200, training_loss=1.0403571283817292, metrics={'train_runtime': 1798.9365, 'train_samples_per_second': 10.673, 'train_steps_per_second': 0.667, 'total_flos': 5051913692774400.0, 'train_loss': 1.0403571283817292, 'epoch': 4.0})

In [ ]:
predictions = trainer.predict(test_dataset)
predicted_labels_indices = predictions.predictions.argmax(-1)

# Assuming the label dictionary used earlier was like this:
label_dict = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
# Invert the dictionary to map back from indices to labels
index_to_label = {v: k for k, v in label_dict.items()}

# Map the predicted label indices to actual labels
predicted_labels = [index_to_label[idx] for idx in predicted_labels_indices]


In [ ]:
result_A2B1C1['difficulty'] = predicted_labels
result_A2B1C1

<ipython-input-113-1333fc080a3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_A2B1C1['difficulty'] = predicted_labels


,id,difficulty,sentence
933,0,C2,Nous dûmes nous excuser des propos que nous eû...
0,1,A2,Vous ne pouvez pas savoir le plaisir que j'ai ...
464,2,B1,"Et, paradoxalement, boire froid n'est pas la b..."
935,5,C1,"Elle jeta un cri, un petit cri, voulut se dres..."
2,6,A1,"Madame, Monsieur, Votre fils Léo arrive tous l..."
...,...,...,...
462,1194,A1,Vous faites de l'exercice physique
930,1195,B1,C'est un phénomène qui trouve une accélération...
463,1196,A2,Je vais parler au serveur et voir si on peut d...
931,1198,B2,Ils deviennent dangereux pour notre économie.


# Final df merge

In [ ]:
df_predictions = pd.concat([result_A1B2C2, result_A2B1C1], ignore_index=True).sort_values(by='id', ascending=True)


In [ ]:
df_predictions = df_predictions[['id', 'difficulty']]
df_predictions

,id,difficulty
0,0,C2
477,1,A2
478,2,B1
1,3,A1
2,4,C2
...,...,...
1197,1195,B1
1198,1196,A2
475,1197,C2
476,1198,B2


In [ ]:
df_predictions.groupby('difficulty').size()

difficulty
A1    216
A2    190
B1    164
B2    317
C1    153
C2    160
dtype: int64

In [ ]:
df_predictions.to_csv('Final_with_ABC_0343_PipelineA1_DistilbertB1B2C1_forB2_Distilbert_C1C2_with_all_data_withDistilbert_forA2B1C1.csv', index=False)

#Test of subsample

In [ ]:
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')

test_B2_difficulty = test_B2[['id', 'difficulty']]  # Ensure 'id' is included for merging
test=test[['id']]
# Now merge the 'test' DataFrame with the 'test_A1_difficulty' DataFrame
test = pd.merge(test, test_B2_difficulty, on='id', how='left')
test['difficulty'] = test['difficulty'].replace('B2', 'D1')
test['difficulty'] = test['difficulty'].replace('B1', 'D1')
test.groupby('difficulty').size()


difficulty
C1     78
D1    398
dtype: int64

In [ ]:
test.to_csv('C2_prediction_Camembert_from_ABC_C1C2.csv', index=False)

In [ ]:
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')

test_C1_difficulty = test_C2[['id', 'difficulty']]  # Ensure 'id' is included for merging
test=test[['id']]
# Now merge the 'test' DataFrame with the 'test_A1_difficulty' DataFrame
test = pd.merge(test, test_C1_difficulty, on='id', how='left')
test['difficulty'] = test['difficulty'].replace('C1', 'D1')
test.groupby('difficulty').size()

difficulty
C2    146
D1    156
dtype: int64